In [1]:
from langchain import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.agents import Tool, tool
from langchain import agents
import langchain
import os
import requests
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import sys
sys.path.append('transformer/')

In [40]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)

paper_id = 1
paper_file =f'../data/papers/{paper_id}.pdf'
pages = PyPDFLoader(paper_file).load_and_split()

sliced_pages = text_splitter.split_documents(pages)
faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
retriever = faiss_vectorstore.as_retriever(search_type='mmr',
        search_kwargs={"k": 20, "fetch_k": 10},
    )

llm=ChatOpenAI(
                model_name='gpt-4',
                temperature=0.1,
                )

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    memory=None, return_source_documents=False
                                )
        
#     # Check to see if we hit the token limit
#     result = qa_chain.run(prompt)
#     return result  

prompt = "What are the design guidelines for making the MOF have a higher band gap?"
def lookup_papers(prompt):
    guide_lines = []
    for m in range(5):
        paper_file =f'../data/papers/{paper_id}.pdf'
        pages = PyPDFLoader(paper_file).load_and_split()
        sliced_pages = text_splitter.split_documents(pages)
        faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
        retriever = faiss_vectorstore.as_retriever(search_type='mmr',
                search_kwargs={"k": 20, "fetch_k": 10},
            )
        
        llm=ChatOpenAI(
                        model_name='gpt-4',
                        temperature=0.1,
                        )
        
        qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                            chain_type="stuff",
                                            retriever=retriever,
                                            memory=None, return_source_documents=False
                                        )
        g = qa_chain({"query": "What are the design guidelines for making the MOF have a higher band gap?"})
        guide_lines.append(g)
        # break
    return guide_lines

guide_lines = lookup_papers(prompt)

In [41]:
guide_lines

[{'query': 'What are the design guidelines for making the MOF have a higher band gap?',
  'result': 'The study suggests that the band gap in Metal-Organic Frameworks (MOFs) can be significantly influenced by modifications to the organic linker and the inorganic cluster in the MOF structure. Specifically, the band gap can be increased by:\n\n1. Hydroxylating the metal center: Hydroxylation of the metal center in the MOF structure increases the band gap. This is because hydroxylation leads to less bound oxygen states, and consequently, a larger band gap'},
 {'query': 'What are the design guidelines for making the MOF have a higher band gap?',
  'result': 'Based on the information provided, the band gap in Metal-Organic Frameworks (MOFs) can be influenced by several factors:\n\n1. Substituting the organic linker: The band gap can be significantly affected by exchanging the organic linker in the MOF. For example, substituting hydrogen with NH2 or NO2 on the linker can reduce the band gap.\

In [38]:
# qa_chain('What is the title of this paper?')
result = qa_chain({"query": "What are the design guidelines for making the MOF have a higher band gap?"})
result

{'query': 'What are the design guidelines for making the MOF have a higher band gap?',
 'result': 'Based on the study, the band gap in Metal-Organic Frameworks (MOFs) can be influenced by several factors:\n\n1. Substituting the organic linker: The band gap can be significantly affected by substituting hydrogen with other elements on the organic linker. For example, substituting hydrogen with NH2 and NO2 on the linker can reduce the band gap.\n\n2. Hydroxylation and Dehydroxylation: The band gap is larger for the hydroxyl'}

In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
Embedding_model = 'text-embedding-ada-002' 

In [42]:
from langchain.agents import Tool, tool
from langchain_community.utilities import GoogleSerperAPIWrapper

langchain.debug = False

@tool
def predict_band_gap(SMILES):
    '''This tool inputs MOF's SMILES string represntations and outputs its band gap.
    '''
    # from ft_model import predictBandGap
    from ft_model_w_uncertainty import predictBandGap
    bg = predictBandGap(SMILES).item()
    return np.round(bg[0], 3), np.round(bg[1], 3)

search = GoogleSerperAPIWrapper(engine='google', serpapi_api_key=os.getenv("SERPER_API_KEY"))

from langchain.agents import load_tools
serpapi = load_tools(["serpapi"])[0]

from langchain.tools import StructuredTool
tools = [predict_band_gap, serpapi]
    #     Tool(
    #     name="Online Search",
    #     func=search.run,
    #     description="useful for when you need to ask with online search",
    # )]

In [43]:
predict_band_gap("[Zn]12.OC(=O)C1=CC=C(C=C1)C(O2)=O")

Running on: cuda:0
Mean predicted band gap is 3.249528932571411 eV
The standard deviation is 0.4893574785507685 eV


AttributeError: 'tuple' object has no attribute 'item'

In [3]:
import langchain
langchain.debug = False


In [4]:
from langchain.agents.structured_chat.base import StructuredChatAgent
from langchain.agents.agent import AgentExecutor
from langchain.agents import AgentType
from langchain.schema import HumanMessage, SystemMessage


message = """You are a helpful Chemist AI assistant. You are tasked to predict the band gap of a given MOF. 
    Use the tools to get chemical intution and if you don't know, do not make up answers.
    """

system_message = SystemMessage(
  content=message
)



from langchain.agents import initialize_agent
agent = initialize_agent(
            tools=tools, llm=llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True, memory=None,
            early_stopping_method='generate', handle_parsing_errors=True, system_message=system_message, prefix=message,
            agent_kwargs = {
        "system_message": system_message,
    }
            )



In [5]:
agent.run(input="What is the band gap of mof [Co].[O-]C(=O)c1ccncc1?")



> Entering new AgentExecutor chain...
Thought: To find the band gap of the MOF, I need to use the predict_band_gap tool with the SMILES string of the MOF as input.
Action:
```
{
  "action": "predict_band_gap",
  "action_input": "[Co].[O-]C(=O)c1ccncc1"
}
```
Observation: 0.468
Thought:Could not parse LLM output: The band gap of the MOF with the given SMILES string is 0.468 eV.
Observation: Invalid or incomplete response
Thought:I now know the final answer
Final Answer: The band gap of the MOF with the SMILES string "[Co].[O-]C(=O)c1ccncc1" is 0.468 eV.

> Finished chain.


'The band gap of the MOF with the SMILES string "[Co].[O-]C(=O)c1ccncc1" is 0.468 eV.'

In [6]:
agent.run(input="Look up design guidlines for mofs with high band gaps,\
            Then bsaed on those, suggest a MOF (SMILES) with high band gap predict its band gap?")



> Entering new AgentExecutor chain...
Thought: First, I need to look up the design guidelines for MOFs with high band gaps. I will use the search tool for this.
Action:
```
{
  "action": "Search",
  "action_input": "design guidelines for MOFs with high band gaps"
}
```
Observation: ['ABSTRACT: Assigning optical band gaps to MOFs is paramount for understanding their optical, electronic, and reactivity properties, but.', 'Here, we report optical band gaps of common MOFs and demonstrate that Gaussian fitting is more appropriate for assigning accurate gap energies.', 'In this work, we have carried out a systematic study to design eight isoreticular MOFs (IR-. MOFs)11 with different kinds of linear linkers (as ...', 'An in-depth and comprehensive understanding of the intrinsic correlation between the structures and band gaps of MOF-based semiconductors will ...', 'The key parameters to be controlled include the band gap, the absolute energy position of band edges, the excited state charge

'The MOF with the SMILES representation "C1=CC=C(C=C1)C2=CC=CC=C2" has a predicted band gap of 1.784 eV, which is considered high.'